In [1]:
import pandas as pd
import numpy as np
import openpyxl
import pandas as pd
import io
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors,KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [4]:
import os
# Get cwd
cwd = os.getcwd()
# cd .. to get to the parent directory
cwd = os.path.dirname(cwd)

#create a dataframe wit example_dt_1.csv
df = pd.read_csv(os.path.join(cwd, 'data', 'flatten_data.csv'))
df

,country,2004_life_exp,2005_life_exp,2006_life_exp,2007_life_exp,2008_life_exp,2009_life_exp,2010_life_exp,2011_life_exp,2012_life_exp,...,2009_pm,2010_pm,2011_pm,2012_pm,2013_pm,2014_pm,2015_pm,2016_pm,2017_pm,2018_pm
0,ABW,73.576,73.811,74.026,74.210,74.147,74.560,75.404,75.465,75.531,...,2.420000,4.820000,3.270000,2.980000,2.590000,2.160000,2.260000,2.990000,2.99,2.400000
1,AFG,57.944,58.361,58.684,59.111,59.852,60.364,60.851,61.419,61.923,...,14.760000,14.780000,18.010000,16.070000,14.640000,15.170000,15.720000,15.770000,14.91,13.710000
2,AGO,50.592,51.570,52.369,53.642,54.633,55.752,56.726,57.596,58.623,...,19.860001,23.549999,21.400000,21.450001,21.400000,18.700001,20.010000,20.120001,18.09,18.790001
3,ALB,76.376,76.621,76.816,77.549,77.653,77.781,77.936,78.092,78.064,...,14.050000,11.500000,19.620001,16.250000,13.560000,12.350000,15.500000,13.040000,14.23,13.680000
4,ARE,76.453,76.683,76.909,77.129,77.488,78.003,78.334,78.517,78.716,...,19.080000,18.540001,18.180000,21.030001,18.860001,18.600000,17.400000,18.680000,18.08,18.139999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,VUT,69.313,69.390,69.462,69.544,69.602,69.653,69.617,69.551,69.497,...,3.500000,3.980000,4.320000,4.040000,4.600000,4.340000,3.950000,4.190000,4.73,4.510000
192,WSM,71.696,71.754,72.021,72.100,72.155,70.066,72.249,72.260,72.167,...,0.950000,0.810000,0.750000,0.910000,0.940000,1.000000,1.260000,1.060000,1.07,1.140000
193,ZAF,54.043,53.980,54.278,54.992,56.022,57.447,58.899,60.651,61.846,...,22.629999,25.700001,22.440001,22.379999,21.260000,20.480000,21.680000,20.209999,21.15,21.219999
194,ZMB,49.487,50.659,51.795,52.739,53.945,55.300,56.799,57.771,58.867,...,21.209999,23.120001,20.200001,21.530001,20.180000,19.650000,21.620001,20.230000,18.92,19.350000


In [21]:
df[df['country']=='IND']

,country,2004_life_exp,2005_life_exp,2006_life_exp,2007_life_exp,2008_life_exp,2009_life_exp,2010_life_exp,2011_life_exp,2012_life_exp,...,2010_pm,2011_pm,2012_pm,2013_pm,2014_pm,2015_pm,2016_pm,2017_pm,2018_pm,class
81,IND,64.524,64.996,65.412,65.788,66.149,66.513,66.909,67.359,67.887,...,52.330002,55.959999,50.970001,53.220001,51.700001,51.619999,55.099998,52.889999,53.450001,5


In [7]:
#create a df with all the columns except the first one
df2 = df.iloc[:,1:]
scale = StandardScaler()
df2_scaled = scale.fit_transform(df2)
print(df2_scaled)

[[ 0.53155813  0.52911649  0.52407554 ... -1.03515557 -1.08892629
  -1.09436388]
 [-1.0894819  -1.09462989 -1.1171023  ...  0.22000118  0.14518348
   0.04082408]
 [-1.85188507 -1.80834263 -1.792636   ...  0.647226    0.47441746
   0.55070525]
 ...
 [-1.49401601 -1.55505921 -1.58842478 ...  0.65606493  0.79122752
   0.79460485]
 [-1.96647368 -1.90408586 -1.85403843 ...  0.65802928  0.5603496
   0.60691251]
 [-2.4834187  -2.52321142 -2.54198098 ...  0.13455624  0.08616984
   0.26063501]]


In [36]:
#Inicialitzem el score més alt a un valor baix
highest_score = -1

#Acoplem el kmeans i calculem el silhouette score per k entre 2 i 15
for k in range(10, 30):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(df2_scaled)
    silhouette_avg = silhouette_score(df2_scaled, kmeans.labels_)
    print("For n_clusters =", k,
          "The average silhouette_score is :", silhouette_avg)
    #Actualitzem el score més alt si el score actual és més alt
    if silhouette_avg > highest_score:
        highest_score = silhouette_avg
        best_k = k

print("The highest silhouette score is:", highest_score)
print("The optimal number of clusters is:", best_k)

C:\Users\reloo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\reloo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\reloo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\U

For n_clusters = 10 The average silhouette_score is : 0.3191604945605212
For n_clusters = 11 The average silhouette_score is : 0.3183031933103464
For n_clusters = 12 The average silhouette_score is : 0.3149865338160149
For n_clusters = 13 The average silhouette_score is : 0.3174231728593136


C:\Users\reloo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\reloo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\reloo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


For n_clusters = 14 The average silhouette_score is : 0.3239760998094003
For n_clusters = 15 The average silhouette_score is : 0.29557310094209543
For n_clusters = 16 The average silhouette_score is : 0.30230370299667775


C:\Users\reloo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\reloo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\reloo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


...

For n_clusters = 24 The average silhouette_score is : 0.2766833657333028
For n_clusters = 25 The average silhouette_score is : 0.289299053209577


C:\Users\reloo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\reloo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


For n_clusters = 26 The average silhouette_score is : 0.29395606948146713
For n_clusters = 27 The average silhouette_score is : 0.2783338553632713


C:\Users\reloo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\reloo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


For n_clusters = 28 The average silhouette_score is : 0.2776930379184019
For n_clusters = 29 The average silhouette_score is : 0.28877143751173634
The highest silhouette score is: 0.3239760998094003
The optimal number of clusters is: 14


In [34]:
#km=KMeans()
#print(km.get_params())
#parameters={'n_clusters':[9,10,11,12,13,14,15,16,17,18,19,20]}
#grid=GridSearchCV(km,parameters,cv=5,scoring='accuracy')
#grid.fit(df2_scaled)
#print(grid.best_params_)


{'algorithm': 'lloyd', 'copy_x': True, 'init': 'k-means++', 'max_iter': 300, 'n_clusters': 8, 'n_init': 'warn', 'random_state': None, 'tol': 0.0001, 'verbose': 0}


C:\Users\reloo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\reloo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\reloo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'



{'n_clusters': 9}


C:\Users\reloo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\reloo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
C:\Users\reloo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
C:\Users\

In [37]:
#Executem el kmeans amb el millor k
kmeans = KMeans(
    init="random",#Inicialitzem els centroids de forma aleatòria
    n_clusters = best_k,#Nombre de clusters = best_k
    n_init=20,#Nombre de vegades que s'executa el kmeans amb centroides diferents
    max_iter=400,#Màxim de iteracions per a cada kmeans
    random_state=1#Seed
)

#Fit the model
kmeans.fit(df2_scaled)

#Resultats
print("n_clusters:",kmeans.n_clusters,"\n")#Nombre de clusters
print("n_iter_:",kmeans.n_iter_,"\n")#Nombre d'iteracions fetes
print("labels\n",kmeans.labels_,"\n")#Etiqueta de cada punt
print("centers\n",kmeans.cluster_centers_,"\n")#Centroids
print("inertia_\n",kmeans.inertia_,"\n")#Suma dels quadrats de les distàncies dels punts als seus centroids

n_clusters: 14 

n_iter_: 11 

labels
 [ 7  9  9  3  3  3  2  4  4  3  2  8  3  9 13 10  0  3  7  0  2  2  4  8
  6  7  7  8  9 11  4  3  3  1 13  8  8  2 12  7  3  4  3  3  3  9  3  7
  7  2  2 12  4  7  9  4  5  3  4  7  9  3  2  9  3 13 12 13  8  3  7  7
  0  4 12  0  0 12  2  6  4  1  4  2  8  4  3  3  2  2  3  2  9  2  2  5
  0  3  8  3 13  7  7  3  0 11  2  3  2  4  7  2 12  3  6  0 13  4  8  2
  8 13  5  7  9  2  9  4 13 11  2  3  4 10  4  3 10  4  0  2 12  0  4  0
  4  2  2  4  0  2  6  8  0 12 12  3  7 13  0 13  0 13  7  0  3  4 13  7
  2  4 13  9  0  2 12 12  7  7  7  0  5  9  8  2  4  6  7  2  7  4  0  7
  7  9  9 13] 

centers
 [[ 0.46109597  0.47266626  0.47933602  0.47029055  0.46734574  0.44683305
   0.47159328  0.47256243  0.47629717  0.48545896  0.48129468  0.48198534
   0.48630183  0.48242081  0.48370549 -0.06845657 -0.06929626 -0.07000565
  -0.07072106 -0.07141092 -0.07212558 -0.07291042 -0.07362804 -0.07446563
  -0.07520851 -0.07581458 -0.07643519 -0.07707886 -0.077

In [38]:
kmeans.labels_
#transform to dataframe
df_labels = pd.DataFrame(kmeans.labels_)
df_labels

,0
0,7
1,9
2,9
3,3
4,3
...,...
191,7
192,7
193,9
194,9


In [39]:
df['class']=df_labels
df

,country,2004_life_exp,2005_life_exp,2006_life_exp,2007_life_exp,2008_life_exp,2009_life_exp,2010_life_exp,2011_life_exp,2012_life_exp,...,2010_pm,2011_pm,2012_pm,2013_pm,2014_pm,2015_pm,2016_pm,2017_pm,2018_pm,class
0,ABW,73.576,73.811,74.026,74.210,74.147,74.560,75.404,75.465,75.531,...,4.820000,3.270000,2.980000,2.590000,2.160000,2.260000,2.990000,2.99,2.400000,7
1,AFG,57.944,58.361,58.684,59.111,59.852,60.364,60.851,61.419,61.923,...,14.780000,18.010000,16.070000,14.640000,15.170000,15.720000,15.770000,14.91,13.710000,9
2,AGO,50.592,51.570,52.369,53.642,54.633,55.752,56.726,57.596,58.623,...,23.549999,21.400000,21.450001,21.400000,18.700001,20.010000,20.120001,18.09,18.790001,9
3,ALB,76.376,76.621,76.816,77.549,77.653,77.781,77.936,78.092,78.064,...,11.500000,19.620001,16.250000,13.560000,12.350000,15.500000,13.040000,14.23,13.680000,3
4,ARE,76.453,76.683,76.909,77.129,77.488,78.003,78.334,78.517,78.716,...,18.540001,18.180000,21.030001,18.860001,18.600000,17.400000,18.680000,18.08,18.139999,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,VUT,69.313,69.390,69.462,69.544,69.602,69.653,69.617,69.551,69.497,...,3.980000,4.320000,4.040000,4.600000,4.340000,3.950000,4.190000,4.73,4.510000,7
192,WSM,71.696,71.754,72.021,72.100,72.155,70.066,72.249,72.260,72.167,...,0.810000,0.750000,0.910000,0.940000,1.000000,1.260000,1.060000,1.07,1.140000,7
193,ZAF,54.043,53.980,54.278,54.992,56.022,57.447,58.899,60.651,61.846,...,25.700001,22.440001,22.379999,21.260000,20.480000,21.680000,20.209999,21.15,21.219999,9
194,ZMB,49.487,50.659,51.795,52.739,53.945,55.300,56.799,57.771,58.867,...,23.120001,20.200001,21.530001,20.180000,19.650000,21.620001,20.230000,18.92,19.350000,9


In [40]:
df.to_csv(os.path.join(cwd, 'data', 'nested_df_kmeans.csv'))